In [1]:
import os
os.chdir('..')

In [11]:
from src.models.decoders import MNISTDecoder
import torch
from torch.distributions import Bernoulli
import torch.nn.functional as F

In [4]:
decoder = MNISTDecoder(z_dim=20)
x = torch.randn(32, 784)
z = torch.randn(32, 20)

In [7]:
loc_img = decoder.decode(z)

In [10]:
dist = Bernoulli(probs=loc_img)
log_px_z_1 = dist.log_prob(x).sum(-1)

In [13]:
log_px_z_2 = -F.binary_cross_entropy(loc_img, x, reduction='none').sum(-1)

In [15]:
log_px_z_1

tensor([-545.2446, -541.9825, -554.2417, -547.8779, -555.0793, -543.4597,
        -540.7529, -550.2977, -552.5751, -547.5850, -546.8231, -544.1868,
        -549.9807, -553.7803, -546.7213, -546.8846, -546.6425, -544.6062,
        -545.1661, -549.4921, -551.7457, -544.6907, -544.5678, -556.7418,
        -543.5345, -545.6293, -539.3209, -554.2197, -546.2762, -550.2283,
        -550.1263, -552.4769], grad_fn=<SumBackward1>)

In [16]:
log_px_z_2

tensor([-545.2446, -541.9825, -554.2417, -547.8779, -555.0793, -543.4597,
        -540.7529, -550.2977, -552.5751, -547.5850, -546.8231, -544.1868,
        -549.9807, -553.7803, -546.7213, -546.8846, -546.6425, -544.6062,
        -545.1661, -549.4921, -551.7457, -544.6907, -544.5678, -556.7418,
        -543.5345, -545.6292, -539.3210, -554.2197, -546.2761, -550.2283,
        -550.1263, -552.4769], grad_fn=<NegBackward>)